In [1]:
from datasets import load_dataset

dataset_name = 'liupf/ChEBI-20-MM'
# Replace 'dataset_name' with the actual name of the dataset you want to load
dataset = load_dataset(dataset_name)
df_train = dataset['train'].to_pandas()


/home/ali.lawati/gnn-incontext/.conda/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import torch_geometric.utils.smiles as smiles
smiles_str = "CCO"
graph = smiles.from_smiles(smiles_str)
#print(graph)
print(graph.x, graph.edge_index)
graph = smiles.from_smiles("OCC")
print(graph.x, graph.edge_index)
graph = smiles.from_smiles("C(O)C")
print(graph.x, graph.edge_index)
#print(graph.edge_attr)

tensor([[6, 0, 4, 5, 3, 0, 4, 0, 0],
        [6, 0, 4, 5, 2, 0, 4, 0, 0],
        [8, 0, 2, 5, 1, 0, 4, 0, 0]]) tensor([[0, 1, 1, 2],
        [1, 0, 2, 1]])
tensor([[8, 0, 2, 5, 1, 0, 4, 0, 0],
        [6, 0, 4, 5, 2, 0, 4, 0, 0],
        [6, 0, 4, 5, 3, 0, 4, 0, 0]]) tensor([[0, 1, 1, 2],
        [1, 0, 2, 1]])
tensor([[6, 0, 4, 5, 2, 0, 4, 0, 0],
        [8, 0, 2, 5, 1, 0, 4, 0, 0],
        [6, 0, 4, 5, 3, 0, 4, 0, 0]]) tensor([[0, 0, 1, 2],
        [1, 2, 0, 0]])


In [3]:
test_graphs = df_train['SMILES'].apply(smiles.from_smiles)
test_graphs[0]

Data(x=[24, 9], edge_index=[2, 48], edge_attr=[48, 3], smiles='CCCCC[C@@H]1O[C@@H]1/C=C/C(O)C/C=C\C/C=C\CCCC(=O)[O-]')

In [4]:
import sys
import os

# Adjust the path to the directory containing VGAE_pyG
sys.path.append(os.path.abspath(os.path.join('..', 'VGAE_pyG')))

from model import DeepVGAE


In [5]:
import os
import torch
import torch.nn as nn
from torch.optim import Adam

from torch_geometric.datasets import Planetoid
import torch_geometric.transforms as T
from torch_geometric.utils import train_test_split_edges

from model import DeepVGAE
from config.config import parse_args
torch.manual_seed(12345)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

#args = parse_args()
#print(args)
class Args:
    def __init__(self, **kwargs):
        self.__dict__.update(kwargs)

# Converting the dictionary to an object
args = Args(
    dataset="Cora",
    enc_in_channels=1433,
    enc_hidden_channels=32,
    enc_out_channels=16,
    lr=0.01,
    epoch=400
)

In [9]:
model = DeepVGAE(args).to(device)
optimizer = Adam(model.parameters(), lr=args.lr)

os.makedirs("datasets", exist_ok=True)
dataset = Planetoid("datasets", args.dataset, transform=T.NormalizeFeatures())
data = dataset[0].to(device)

data

Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])

In [ ]:
all_edge_index = data.edge_index
data = train_test_split_edges(data, 0.05, 0.1)

for epoch in range(args.epoch):
    model.train()
    optimizer.zero_grad()
    loss = model.loss(data.x, data.train_pos_edge_index, all_edge_index)
    loss.backward()
    optimizer.step()
    if epoch % 2 == 0:
        model.eval()
        roc_auc, ap = model.single_test(data.x,
                                        data.train_pos_edge_index,
                                        data.test_pos_edge_index,
                                        data.test_neg_edge_index)
        print("Epoch {} - Loss: {} ROC_AUC: {} Precision: {}".format(epoch, loss.cpu().item(), roc_auc, ap))